## 1. 📦 Install Dependencies

In [ ]:
!pip install -q transformers einops h5py

## 2. 📂 Data Paths (Kaggle Input)

In [ ]:
# Data paths từ Kaggle Input
text_feature_path = '/kaggle/input/text-feature'
visual_feature_path = '/kaggle/input/visual-feature'
split_path = '/kaggle/input/dataset-split-1'
text_annotation_path = '/kaggle/input/text-annotation'

print("✅ Data paths:")
print(f"  Text features: {text_feature_path}")
print(f"  Visual features: {visual_feature_path}")
print(f"  Split files: {split_path}")
print(f"  Text annotations: {text_annotation_path}")

# Verify paths exist
import os
for path in [text_feature_path, visual_feature_path, split_path, text_annotation_path]:
    if os.path.exists(path):
        print(f"  ✓ {path} exists")
    else:
        print(f"  ✗ {path} NOT FOUND!")

## 3. ⚙️ Configuration

In [ ]:
import os
import sys
import torch
import numpy as np

# ============================================================
# TRAINING CONFIGURATION
# ============================================================

class Config:
    # Version name
    v = "causalvid_kaggle"
    
    # Data paths (from Kaggle input)
    sample_list_path = split_path
    video_feature_path = visual_feature_path
    text_annotation_path = text_annotation_path
    
    # Training params
    bs = 8                     # Batch size (reduce if OOM)
    lr = 1e-5                  # Learning rate
    epoch = 5                  # Number of epochs
    gpu = 0                    # GPU ID
    
    # Dataset params
    dataset = 'causal-vid'
    qtype = -1                 # -1 for all, 0-5 for specific
    max_samples = 20           # 🔥 Giới hạn số video (20 videos = 120 samples với qtype=-1)
    objs = 20                  # Number of objects
    n_query = 5                # Number of answer choices
    
    # Model params
    d_model = 768
    word_dim = 768
    topK_frame = 8             # Top-K frames
    topK_obj = 5               # Top-K objects
    hard_eval = False
    frame_feat_dim = 4096      # app + mot concatenated
    obj_feat_dim = 2053        # 2048 + 5 bbox
    
    # Transformer params
    num_encoder_layers = 1
    num_decoder_layers = 1
    nheads = 8
    normalize_before = False
    activation = 'relu'
    
    # Dropout
    dropout = 0.1
    encoder_dropout = 0.1
    
    # Text encoder
    text_encoder_type = "microsoft/deberta-base"
    text_encoder_lr = 5e-6
    freeze_text_encoder = False
    text_pool_mode = 0
    
    # Scheduler
    patience = 1
    gamma = 0.25
    decay = 0.001
    
    # Contrastive learning
    pos_ratio = 0.7
    neg_ratio = 0.3
    a = 1
    
    # Early stopping
    es = False

args = Config()
print(f"✅ Config loaded: {args.v}")
print(f"   Batch size: {args.bs}")
print(f"   Epochs: {args.epoch}")
print(f"   Max samples: {args.max_samples} videos ({args.max_samples * 6} samples với qtype=-1)")

## 4. 📚 Import Modules

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
import json

# Import local modules
from DataLoader import VideoQADataset
from networks.model import VideoQAmodel
import eval_mc

# Set seed for reproducibility
def set_seed(seed=999):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(999)
print("✅ Modules imported")

## 5. 📊 Create DataLoaders

In [ ]:
print("Creating datasets...")

# Train dataset
train_dataset = VideoQADataset(
    split='train',
    n_query=args.n_query,
    obj_num=args.objs,
    sample_list_path=args.sample_list_path,
    video_feature_path=args.video_feature_path,
    text_annotation_path=args.text_annotation_path,
    qtype=args.qtype,
    max_samples=args.max_samples
)

# Validation dataset
val_dataset = VideoQADataset(
    split='val',
    n_query=args.n_query,
    obj_num=args.objs,
    sample_list_path=args.sample_list_path,
    video_feature_path=args.video_feature_path,
    text_annotation_path=args.text_annotation_path,
    qtype=args.qtype,
    max_samples=args.max_samples
)

# Test dataset
test_dataset = VideoQADataset(
    split='test',
    n_query=args.n_query,
    obj_num=args.objs,
    sample_list_path=args.sample_list_path,
    video_feature_path=args.video_feature_path,
    text_annotation_path=args.text_annotation_path,
    qtype=args.qtype,
    max_samples=args.max_samples
)

# Create dataloaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=args.bs,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=args.bs,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=args.bs,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print(f"\n✅ DataLoaders created:")
print(f"   Train: {len(train_dataset)} samples, {len(train_loader)} batches")
print(f"   Val: {len(val_dataset)} samples, {len(val_loader)} batches")
print(f"   Test: {len(test_dataset)} samples, {len(test_loader)} batches")

## 6. 🔍 Check Data Sample

In [ ]:
# Check one batch
for batch in train_loader:
    vid_frame_feat, vid_obj_feat, qns_word, ans_word, ans_id, qns_key = batch
    print("Sample batch:")
    print(f"  Frame feat shape: {vid_frame_feat.shape}")
    print(f"  Object feat shape: {vid_obj_feat.shape}")
    print(f"  Question: {qns_word[0]}")
    print(f"  Answer choices: {ans_word[0][0][:100]}...")
    print(f"  Answer ID: {ans_id[0]}")
    print(f"  Question key: {qns_key[0]}")
    break

## 7. 🏗️ Create Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Convert args to dict for model
args.device = device
config = vars(args)

# Create model
model = VideoQAmodel(**config)
model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n✅ Model created:")
print(f"   Total parameters: {total_params/1e6:.2f}M")
print(f"   Trainable parameters: {trainable_params/1e6:.2f}M")

## 8. 🎯 Training Functions

In [ ]:
# GradScaler for mixed precision
scaler = torch.cuda.amp.GradScaler(enabled=True)

def train_epoch(model, optimizer, train_loader, criterion, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0
    prediction_list = []
    answer_list = []
    
    for batch_idx, inputs in enumerate(train_loader):
        vid_frame_feat, vid_obj_feat, qns_w, ans_w, ans_id, _ = inputs
        vid_frame_feat = vid_frame_feat.to(device)
        vid_obj_feat = vid_obj_feat.to(device)
        ans_targets = ans_id.to(device)
        
        with torch.cuda.amp.autocast(enabled=True):
            out = model(vid_frame_feat, vid_obj_feat, qns_w, ans_w)
            loss = criterion(out, ans_targets)
        
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        prediction = out.max(-1)[1]
        prediction_list.append(prediction)
        answer_list.append(ans_id)
        
        if (batch_idx + 1) % 50 == 0:
            print(f"    Batch [{batch_idx+1}/{len(train_loader)}] Loss: {loss.item():.4f}")
    
    predict_answers = torch.cat(prediction_list, dim=0).long().cpu()
    ref_answers = torch.cat(answer_list, dim=0).long()
    acc = (predict_answers == ref_answers).sum().item() * 100.0 / len(ref_answers)
    
    return total_loss / len(train_loader), acc


def evaluate(model, data_loader, device):
    """Evaluate on validation/test set"""
    model.eval()
    prediction_list = []
    answer_list = []
    
    with torch.no_grad():
        for inputs in data_loader:
            vid_frame_feat, vid_obj_feat, qns_w, ans_w, ans_id, _ = inputs
            vid_frame_feat = vid_frame_feat.to(device)
            vid_obj_feat = vid_obj_feat.to(device)
            
            out = model(vid_frame_feat, vid_obj_feat, qns_w, ans_w)
            prediction = out.max(-1)[1]
            prediction_list.append(prediction)
            answer_list.append(ans_id)
    
    predict_answers = torch.cat(prediction_list, dim=0).long().cpu()
    ref_answers = torch.cat(answer_list, dim=0).long()
    acc = (predict_answers == ref_answers).sum().item() * 100.0 / len(ref_answers)
    
    return acc


def predict_and_save(model, data_loader, device, save_path):
    """Predict and save results"""
    model.eval()
    results = {}
    prediction_list = []
    answer_list = []
    
    with torch.no_grad():
        for inputs in data_loader:
            vid_frame_feat, vid_obj_feat, qns_w, ans_w, ans_id, qns_keys = inputs
            vid_frame_feat = vid_frame_feat.to(device)
            vid_obj_feat = vid_obj_feat.to(device)
            
            out = model(vid_frame_feat, vid_obj_feat, qns_w, ans_w)
            prediction = out.max(-1)[1]
            prediction_list.append(prediction)
            answer_list.append(ans_id)
            
            for qid, pred, ans in zip(qns_keys, prediction.cpu().numpy(), ans_id.numpy()):
                results[qid] = {'prediction': int(pred), 'answer': int(ans)}
    
    predict_answers = torch.cat(prediction_list, dim=0).long().cpu()
    ref_answers = torch.cat(answer_list, dim=0).long()
    acc = (predict_answers == ref_answers).sum().item() * 100.0 / len(ref_answers)
    
    # Save results
    with open(save_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    return results, acc

print("✅ Training functions defined")

## 9. 🚀 Training Loop

In [ ]:
# Create output directories
os.makedirs('./models', exist_ok=True)
os.makedirs('./prediction', exist_ok=True)

# Optimizer with different LR for text encoder
param_dicts = [
    {"params": [p for n, p in model.named_parameters() if "text_encoder" not in n and p.requires_grad]},
    {"params": [p for n, p in model.named_parameters() if "text_encoder" in n and p.requires_grad], 
     "lr": args.text_encoder_lr}
]
optimizer = torch.optim.AdamW(params=param_dicts, lr=args.lr, weight_decay=args.decay)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=args.gamma, patience=args.patience, verbose=True)
criterion = nn.CrossEntropyLoss().to(device)

print("✅ Optimizer and scheduler created")
print(f"   Main LR: {args.lr}")
print(f"   Text encoder LR: {args.text_encoder_lr}")

In [ ]:
# Training
best_val_acc = 0.0
best_epoch = 1
best_model_path = f'./models/best_model-{args.v}.ckpt'

print("="*60)
print(f"🚀 Starting training: {args.v}")
print(f"   Epochs: {args.epoch}")
print(f"   Batch size: {args.bs}")
print("="*60)

for epoch in range(1, args.epoch + 1):
    start_time = time.time()
    
    # Train
    print(f"\n📚 Epoch [{epoch}/{args.epoch}]")
    train_loss, train_acc = train_epoch(model, optimizer, train_loader, criterion, device)
    
    # Evaluate
    val_acc = evaluate(model, val_loader, device)
    test_acc = evaluate(model, test_loader, device)
    
    # Update scheduler
    scheduler.step(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        torch.save(model.state_dict(), best_model_path)
        print(f"   💾 Saved best model!")
    
    elapsed = time.time() - start_time
    print(f"   ⏱️ Time: {elapsed:.1f}s")
    print(f"   📉 Loss: {train_loss:.4f}")
    print(f"   🎯 Train Acc: {train_acc:.2f}%")
    print(f"   🎯 Val Acc: {val_acc:.2f}% {'⭐ BEST' if val_acc >= best_val_acc else ''}")
    print(f"   🎯 Test Acc: {test_acc:.2f}%")

print("\n" + "="*60)
print(f"✅ Training completed!")
print(f"   Best epoch: {best_epoch}")
print(f"   Best val acc: {best_val_acc:.2f}%")
print("="*60)

## 10. 📊 Final Evaluation

In [ ]:
# Load best model
print("Loading best model...")
model.load_state_dict(torch.load(best_model_path))

# Predict on test set
result_path = f'./prediction/{args.v}-{best_epoch}-{best_val_acc:.2f}.json'
results, test_acc = predict_and_save(model, test_loader, device, result_path)

print(f"\n📊 Final Test Results:")
print(f"   Overall Accuracy: {test_acc:.2f}%")
print(f"   Results saved to: {result_path}")

In [ ]:
# Detailed evaluation by question type
print("\n📊 Accuracy by Question Type:")
print("="*60)
eval_mc.accuracy_metric_cvid(result_path)

## 11. 💾 Save Model to Kaggle Output

In [ ]:
# Copy model to /kaggle/working for download
import shutil

output_dir = '/kaggle/working'
if os.path.exists(output_dir):
    # Copy best model
    shutil.copy(best_model_path, os.path.join(output_dir, f'best_model-{args.v}.ckpt'))
    # Copy predictions
    shutil.copy(result_path, output_dir)
    print(f"✅ Files saved to {output_dir}")
else:
    print("Not running on Kaggle, files saved locally")

---

## 🔧 Quick Test (Optional)

Uncomment và chạy cell dưới để test nhanh với ít data:

In [ ]:
# # Quick test với 10 videos
# args.max_samples = 10
# args.bs = 4
# args.epoch = 2
# 
# # Re-create datasets với max_samples
# train_dataset = VideoQADataset(
#     split='train', n_query=args.n_query, obj_num=args.objs,
#     sample_list_path=args.sample_list_path,
#     video_feature_path=args.video_feature_path,
#     text_annotation_path=args.text_annotation_path,
#     qtype=args.qtype, max_samples=args.max_samples
# )
# print(f"Quick test dataset: {len(train_dataset)} samples")